<a href="https://colab.research.google.com/github/sollall/myidea/blob/master/%E5%88%86%E5%AD%90%E9%87%8F%E5%88%86%E5%B8%83_FFT_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
import cmath
 
class FFT:
    def __init__(self):
        self.pi = cmath.pi
    
    def convolution(self, g, h):
        l = len(g) + len(h)
        n = 1 << l.bit_length()
        
        g = g + [0] * (n - len(g))
        h = h + [0] * (n - len(h))
       
        gg = self.fft(g, n)
        hh = self.fft(h, n)
        ff = [gg[i] * hh[i] for i in range(n)]
 
        return self.inv_fft(ff, n)
 
    def fft(self, f, n, sgn = 1):
        if n == 1: return f
 
        f0 = [f[2 * i] for i in range(n // 2)]
        f1 = [f[2 * i + 1] for i in range(n // 2)]
        f0 = self.fft(f0, n // 2, sgn)
        f1 = self.fft(f1, n // 2, sgn)
 
        zeta = cmath.rect(1, sgn * self.pi * 2 / n)
        pow_zeta = 1
 
        for i in range(n):
            f[i] = f0[i % (n // 2)] + pow_zeta * f1[i % (n // 2)]
            pow_zeta *= zeta
 
        return f
 
    def inv_fft(self, f, n):
        ret = self.fft(f, n, -1)
        for i in range(n):
            ret[i] = (ret[i].real / n)
 
        return ret

In [ ]:
"""
方針
モノマーのみと反応させる　不活化したポリマーは再活性化しない(別に話が簡単になっていいけど)
モノマーとラジカルの重合確率は同じとする
"""

Time=3000
N=3*10**4
mol=10**6
init=mol/100#

mono=[0 for i in range(2)] #モノマーだけ
mono[1]=mol

#ループ開始用
A=mono[:]+[0]*(N-len(mono))
B=[0.0]*N
C=[0.0]*N

fft=FFT()

k=0
r=0
threshold=init*k

for step in range(Time):
    print("開始剤:%f" % (init))
    k+=0.00001
    #k*r_initはランダムでラジカル化→k*init*A
    delta_init=k*init
    init-=delta_init
    init=max(init,0)
    B[0]+=delta_init
    
    print(step)
    #FFTでアタックする確率を求める　今は単に存在確率としてる
    SUM_A=sum(A)
    SUM_B=sum(B)
    A_per=A[:]
    B_per=B[:]

    A_per=[A_per[i] for i in range(len(A_per))]
    B_per=[B_per[i] for i in range(len(B_per))]

    SUM_A_per=sum(A_per)
    SUM_B_per=sum(B_per)

    A_per=[A_per[i]/(SUM_A_per+SUM_B_per) for i in range(len(A_per))]
    B_per=[B_per[i]/(SUM_A_per+SUM_B_per) for i in range(len(B_per))]


    #FFTする　これはそれぞれの生成を考えるときに使う？
    delta_C=(fft.convolution(B_per,B))
    delta_B=(fft.convolution(A_per,B))

    hell=sum(delta_B)
    print("sumB:%f,sumdeltaB:%f" %(SUM_B,hell))
    
    #ポリマー　前ステップの数-開始剤の攻撃+delta_A/2-delta_B delta_B[i]がおかしい気がする A[i]をA_percent[i]*SUM_Bしないといけない?既存ラジカルの攻撃のため
    A=[A[i] - A_per[i]*SUM_B for i in range(len(A))]
    #ラジカル　衝突による減少+開始剤乖離の発生
    B=[delta_B[i] for i in range(len(A))]
    #ポリマー　
    C=[delta_C[i] for i in range(len(A))]
    
    print('ポリマー量:%f' % (sum([A[i]*i for i in range(len(A))])+sum([B[i]*i for i in range(len(B))])+sum([C[i]*i for i in range(len(C))])))

    print('残りモノマー数:%f' % (sum(A)))
    plt.plot([C[i]*i for i in range(len(A))])
    plt.show()
    plt.plot([B[i]*i for i in range(len(B))])
    plt.show()
    

In [ ]:
Time=3000
N=3*10**4
mol=100000
init=1000#

mono=[0 for i in range(N)]
mono[1]=mol

#ループ開始用
A=mono[:]
B=[0.0]*len(A)

fft=FFT()

k=1
r=0
threshold=init*k

for step in range(Time):
    print(step)
    #k*r_initはランダムでラジカル化→k*init*A
    delta_init=k*init
    init-=delta_init
    init=max(init,0)
    print(init)
    B[0]+=delta_init
    
    #FFTで生成される分子の個数を求める
    SUM_A=sum(A)
    SUM_B=sum(B)
    A_percent=[A[i]/(SUM_A+SUM_B) for i in range(len(A))]
    B_percent=[B[i]/(SUM_A+SUM_B) for i in range(len(B))]

    #FFTする　これはそれぞれの生成を考えるときに使う？
    delta_A=(fft.convolution(B_percent,B))
    delta_B=(fft.convolution(A_percent,B))

    #消費される量は各重合度の分子数の多い順で振り分けるべき　Aはたぶんsum(B)とA_percentから消費量出せる
    SUM_A=sum(A)
    SUM_B=sum(B)
    SUM=SUM_A+SUM_B

    #ラジカルBは(M)/(M+r)*B分、モノマーに戻るのはr/(M+r)*B分
    #delta_AはA/(A+B)、delta_BはB/(A+B)の係数をかける？
    A_percent=[A[i]/(SUM_A+SUM_B) for i in range(len(A))]
    
    #ポリマー　前ステップの数-開始剤の攻撃+delta_A/2-delta_B delta_B[i]がおかしい気がする A[i]をA_percent[i]*SUM_Bしないといけない?既存ラジカルの攻撃のため
    A=[A[i] - (SUM_A)/SUM*A_percent[i]*SUM_B + delta_A[i]/2 for i in range(len(A))]
    #ラジカル　衝突による減少+開始剤乖離の発生
    B=[delta_B[i] for i in range(len(A))]

    print(sum(B))
    print(sum(A[i]*i for i in range(len(A))))
    print(sum(B[i]*i for i in range(len(B))))
    print(A[:30])
    print(B[:30])
    print(A_percent[:30])
    
    

    plt.plot([A[i]*i for i in range(len(A))])
    plt.show()
    plt.plot([B[i]*i for i in range(len(B))])
    plt.show()
    